In [77]:
# Importing necessary libraries
import pandas as pd
import numpy as np

In [78]:
# Loading the CSV files for each year
data_2021 = pd.read_csv("../data/student_responses_2021.csv") 
data_2022 = pd.read_csv("../data/student_responses_2022.csv")  

In [79]:
# Adding a 'year' column to differentiate data from 2021 and 2022
data_2021['year'] = 2021
data_2022['year'] = 2022

# Concatenating both datasets
data = pd.concat([data_2021, data_2022], ignore_index=True)

# Displaying the first few rows
print(data.head())

# Checking data types and missing values
print(data.info())

   student_id  question_id   ability  difficulty  answered_correctly  year
0         967           40 -0.320326    1.572793               False  2021
1         135            2 -2.897524   -1.920777               False  2021
2         553           18 -1.207800   -0.482901               False  2021
3         564           29 -2.012802    0.360224               False  2021
4         449           42  0.332158    1.211066               False  2021
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   student_id          95000 non-null  int64  
 1   question_id         95000 non-null  int64  
 2   ability             93100 non-null  float64
 3   difficulty          93100 non-null  float64
 4   answered_correctly  95000 non-null  bool   
 5   year                95000 non-null  int64  
dtypes: bool(1), float64(2), int64(3)
memory usage

In [80]:
# Checking for any null values in each column
print("Missing values per column:")
print(data.isnull().sum())

Missing values per column:
student_id               0
question_id              0
ability               1900
difficulty            1900
answered_correctly       0
year                     0
dtype: int64


In [81]:
data.describe()

,student_id,question_id,ability,difficulty,year
count,95000.000000,95000.000000,93100.000000,93100.000000,95000.000000
mean,1026.815789,24.315789,0.154100,0.034364,2021.526316
std,576.753180,13.822452,2.326674,1.120549,0.499310
min,1.000000,1.000000,-5.751698,-2.157522,2021.000000
25%,528.000000,12.000000,-1.277524,-0.870725,2021.000000
50%,1050.500000,24.000000,0.031228,-0.147794,2022.000000
75%,1525.250000,36.000000,1.559546,1.069916,2022.000000
max,2000.000000,50.000000,9.957734,2.194765,2022.000000


In [82]:
# Creating quantile-based categories for ability and difficulty
data['ability_range'] = pd.qcut(data['ability'], q=4, duplicates='drop')
data['difficulty_range'] = pd.qcut(data['difficulty'], q=4, duplicates='drop')

In [83]:
data

,student_id,question_id,ability,difficulty,answered_correctly,year,ability_range,difficulty_range
0,967,40,-0.320326,1.572793,False,2021,"(-1.278, 0.0312]","(1.07, 2.195]"
1,135,2,-2.897524,-1.920777,False,2021,"(-5.753, -1.278]","(-2.159, -0.871]"
2,553,18,-1.207800,-0.482901,False,2021,"(-1.278, 0.0312]","(-0.871, -0.148]"
3,564,29,-2.012802,0.360224,False,2021,"(-5.753, -1.278]","(-0.148, 1.07]"
4,449,42,0.332158,1.211066,False,2021,"(0.0312, 1.56]","(1.07, 2.195]"
...,...,...,...,...,...,...,...,...
94995,1402,13,0.460793,-1.077263,True,2022,"(0.0312, 1.56]","(-2.159, -0.871]"
94996,1954,19,4.303861,-0.319677,True,2022,"(1.56, 9.958]","(-0.871, -0.148]"
94997,1763,39,-0.644500,0.947210,False,2022,"(-1.278, 0.0312]","(-0.148, 1.07]"
94998,1632,4,NaN,-1.316538,True,2022,NaN,"(-2.159, -0.871]"


In [63]:
# Checking non-missing values for each group in 'difficulty'
print("Non-missing difficulty values by ability quantile and answered_correctly:")
print(data.groupby(['ability_quantile', 'answered_correctly'])['difficulty'].apply(lambda x: x.notnull().sum()))

# Checking non-missing values for each group in 'ability'
print("\nNon-missing ability values by difficulty quantile and answered_correctly:")
print(data.groupby(['difficulty_quantile', 'answered_correctly'])['ability'].apply(lambda x: x.notnull().sum()))


Non-missing difficulty values by ability quantile and answered_correctly:
ability_quantile  answered_correctly
0.0               False                 22729
                  True                    547
1.0               False                 15227
                  True                   8071
2.0               False                  7120
                  True                  16177
3.0               False                   307
                  True                  22922
Name: difficulty, dtype: int64

Non-missing ability values by difficulty quantile and answered_correctly:
difficulty_quantile  answered_correctly
0.0                  False                  5790
                     True                  17745
1.0                  False                  9140
                     True                  14373
2.0                  False                 13845
                     True                   9689
3.0                  False                 16617
                     True       

In [84]:
# Filling missing values in `difficulty` by considering `year`, `ability_range`, and `answered_correctly`
for (year, ability_range), group in data.groupby(['year', 'ability_range']):
    for correct_status, sub_group in group.groupby('answered_correctly'):
        # Calculating mean difficulty for this subgroup
        mean_difficulty = sub_group['difficulty'].mean()
        # Filling missing values in difficulty within this subgroup
        data.loc[(data['year'] == year) &
                 (data['ability_range'] == ability_range) & 
                 (data['answered_correctly'] == correct_status) & 
                 (data['difficulty'].isnull()), 'difficulty'] = mean_difficulty

C:\Users\Rmkmr\AppData\Local\Temp\ipykernel_15328\3657974662.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (year, ability_range), group in data.groupby(['year', 'ability_range']):


In [85]:
# Filling missing values in `ability` by considering `year`, `difficulty_range`, and `answered_correctly`
for (year, difficulty_range), group in data.groupby(['year', 'difficulty_range']):
    for correct_status, sub_group in group.groupby('answered_correctly'):
        # Calculating mean ability for this subgroup
        mean_ability = sub_group['ability'].mean()
        # Filling missing values in ability within this subgroup
        data.loc[(data['year'] == year) &
                 (data['difficulty_range'] == difficulty_range) & 
                 (data['answered_correctly'] == correct_status) & 
                 (data['ability'].isnull()), 'ability'] = mean_ability

C:\Users\Rmkmr\AppData\Local\Temp\ipykernel_15328\3334684020.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for (year, difficulty_range), group in data.groupby(['year', 'difficulty_range']):


# Note: This code fills missing values in the difficulty and ability columns based on specific subgroup analyses. For difficulty, it calculates the mean difficulty for each combination of year and ability_range, segmented by whether the question was answered correctly or incorrectly. The missing values in difficulty are then filled with these calculated means.Similarly, for ability, the code computes the mean ability for each combination of year and difficulty_range, again considering the correctness of the answers. Missing values in the ability column are filled with these means. This approach ensures that the imputation is informed by the underlying patterns in the data, thereby preserving the relationships between ability, difficulty, and correctness.

In [86]:
# Checking for remaining missing values
print("Missing values after year-based imputation:")
print(data.isnull().sum())

Missing values after year-based imputation:
student_id               0
question_id              0
ability                 48
difficulty              48
answered_correctly       0
year                     0
ability_range         1900
difficulty_range      1900
dtype: int64


In [87]:
# Dropping the extra columns used for imputation
data.drop(columns=['ability_range', 'difficulty_range'], inplace=True)

# Verifying the data structure after dropping the columns
print("Data structure after dropping extra columns:")
print(data.info())

Data structure after dropping extra columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   student_id          95000 non-null  int64  
 1   question_id         95000 non-null  int64  
 2   ability             94952 non-null  float64
 3   difficulty          94952 non-null  float64
 4   answered_correctly  95000 non-null  bool   
 5   year                95000 non-null  int64  
dtypes: bool(1), float64(2), int64(3)
memory usage: 3.7 MB
None


In [88]:
# Dropping rows with any remaining missing values
data.dropna(inplace=True)

# Verifying if all missing values are removed
print("Missing values after dropping remaining rows:")
print(data.isnull().sum())


Missing values after dropping remaining rows:
student_id            0
question_id           0
ability               0
difficulty            0
answered_correctly    0
year                  0
dtype: int64


In [89]:
# Verifying the data structure after dropping the columns
print("Data structure after dropping rows with null values:")
print(data.info())

Data structure after dropping rows with null values:
<class 'pandas.core.frame.DataFrame'>
Index: 94952 entries, 0 to 94999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   student_id          94952 non-null  int64  
 1   question_id         94952 non-null  int64  
 2   ability             94952 non-null  float64
 3   difficulty          94952 non-null  float64
 4   answered_correctly  94952 non-null  bool   
 5   year                94952 non-null  int64  
dtypes: bool(1), float64(2), int64(3)
memory usage: 4.4 MB
None


In [90]:
# Saving the preprocessed data to a CSV file
data.to_csv('../data/merged_and_filled_data.csv', index=False)
print("Merged and Filled data saved successfully.")

Merged and Filled data saved successfully.


In [92]:
# Convert 'answered_correctly' column to binary values
data['answered_correctly'] = data['answered_correctly'].astype(int)

# Check the unique values to confirm the conversion
print(data['answered_correctly'].unique())

[0 1]


In [93]:
data

,student_id,question_id,ability,difficulty,answered_correctly,year
0,967,40,-0.320326,1.572793,0,2021
1,135,2,-2.897524,-1.920777,0,2021
2,553,18,-1.207800,-0.482901,0,2021
3,564,29,-2.012802,0.360224,0,2021
4,449,42,0.332158,1.211066,0,2021
...,...,...,...,...,...,...
94995,1402,13,0.460793,-1.077263,1,2022
94996,1954,19,4.303861,-0.319677,1,2022
94997,1763,39,-0.644500,0.947210,0,2022
94998,1632,4,1.214089,-1.316538,1,2022


In [94]:
correctness_rate = data.groupby('student_id')['answered_correctly'].mean().reset_index()
correctness_rate.columns = ['student_id', 'correctness_rate']
data = data.merge(correctness_rate, on='student_id', how='left')

# Calculating the correctness rate for each student, which could help in assessing performance over time.

In [95]:
attempts_count = data.groupby('student_id')['question_id'].count().reset_index()
attempts_count.columns = ['student_id', 'attempts_count']
data = data.merge(attempts_count, on='student_id', how='left')

# Counting the number of attempts each student has made. This can give insight into how engaged they are with the questions.

In [96]:
data['adjusted_ability'] = data['ability'] - data['difficulty']

# This can provide a more nuanced view of a student's ability relative to the questions they faced.

In [97]:
data

,student_id,question_id,ability,difficulty,answered_correctly,year,correctness_rate,attempts_count,adjusted_ability
0,967,40,-0.320326,1.572793,0,2021,0.422222,45,-1.893119
1,135,2,-2.897524,-1.920777,0,2021,0.000000,45,-0.976748
2,553,18,-1.207800,-0.482901,0,2021,0.177778,45,-0.724899
3,564,29,-2.012802,0.360224,0,2021,0.022222,45,-2.373026
4,449,42,0.332158,1.211066,0,2021,0.622222,45,-0.878908
...,...,...,...,...,...,...,...,...,...
94947,1402,13,0.460793,-1.077263,1,2022,0.580000,50,1.538057
94948,1954,19,4.303861,-0.319677,1,2022,1.000000,50,4.623538
94949,1763,39,-0.644500,0.947210,0,2022,0.320000,50,-1.591710
94950,1632,4,1.214089,-1.316538,1,2022,0.720000,50,2.530628


In [98]:
data.to_csv('../data/preprocessed_data.csv', index = False)